In [1]:
import os
import pandas as pd
import re

train_data_path ="train.csv"
train_data = pd.read_csv(train_data_path)
train_data.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [2]:
# 한국어 형태소 분석기
from konlpy.tag import Okt

okt = Okt()

In [3]:
# 문장 단위 텍스트 전처리
def preprocess_sentence(sentence):
    # 소문자 변환
    sentence = sentence.lower()

    # 문장 부호 제거
    #sentence = re.sub(r"([?.!,\n])", "", sentence)

    # 줄바꿈 문자를 문장 종료 태그 변환
    #sentence = re.sub(r'[\n]', "<eos>", sentence)

    # 문장 부호 간 띄어쓰기
    #sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    
    # 숫자 간 띄어쓰기
    #sentence = re.sub(r"([0-9]+)", r" \1 ", sentence)
    
    # 알파벳 한글 간 띄어쓰기
    #sentence = re.sub(r"([a-zA-Z]+)", r" \1 ", sentence)

    # 알파벳, 한글, 숫자, 문장부호를 제외한 모든 문자를 공백으로 대체
    #sentence = re.sub("[^a-zA-Z0-9가-힣.?!,]", " ", sentence)

    # 빈 칸 반복되는 것 하나로 줄이기
    sentence = re.sub(r'[" "]+', " ", sentence)

    # 문장 앞 뒤 공백 자르기
    sentence = sentence.strip()

    return sentence

In [4]:
# 데이터셋 단위 전처리
def preprocess_data(data):
    res = []
    for conversation in data['conversation']:
        res.append(preprocess_sentence(conversation))

    return res

In [5]:
# 형태소 분리 토큰화
def tokenize(conversations):  
    tokens = []
    for conversation in conversations:
        # 문장 단위로 분리
        sentences = conversation.split('\n')
        token_sentence = []
        for sentence in sentences:
            # 문장 별로 형태소 분석 후 배열에 추가
            for element in okt.pos(sentence):
                token_sentence.append(element)
        tokens.append(token_sentence)
    return tokens

In [12]:
tokenized_conversation = tokenize(train_data['conversation'])
print(len(tokenized_conversation))

3950


In [8]:
# 최대 길이 구하기
max_length = 0
len_tokens = []
for t in tokenized_conversation:
    if (len(t) > max_length):
        max_length = len(t)
    len_tokens.append(len(t))

print(f'토큰화된 문장의 최대 길이: {max_length}')

토큰화된 문장의 최대 길이: 348


In [9]:
print(tokenized_conversation[123])

[('엄마', 'Noun'), ('나도', 'Verb'), ('바람막이', 'Noun'), ('하나', 'Noun'), ('사줘', 'Verb'), ('바람막이', 'Noun'), ('한', 'Verb'), ('삼만원', 'Noun'), ('이', 'Suffix'), ('면', 'Josa'), ('되니', 'Verb'), ('?', 'Punctuation'), ('무슨', 'Noun'), ('삼만원', 'Noun'), ('이야', 'Josa'), ('엄마', 'Noun'), ('는', 'Josa'), ('도데', 'Noun'), ('체', 'Noun'), ('언', 'Modifier'), ('제적', 'Noun'), ('이야기', 'Noun'), ('를', 'Josa'), ('하는거야', 'Verb'), ('엄마', 'Noun'), ('랑', 'Josa'), ('이야기', 'Noun'), ('싫다고', 'Adjective'), ('자꾸', 'Noun'), ('이러니까', 'Conjunction'), ('미안하다', 'Adjective'), ('.', 'Punctuation'), ('엄마', 'Noun'), ('가', 'Josa'), ('잘', 'Verb'), ('몰라서', 'Verb'), ('그래', 'Adjective'), ('아', 'Exclamation'), ('30', 'Number'), ('은', 'Noun'), ('줘야', 'Verb'), ('입을만한', 'Adjective'), ('거', 'Noun'), ('사', 'Noun'), ('30', 'Number'), ('이나', 'Noun'), ('?', 'Punctuation'), ('이번', 'Noun'), ('달', 'Noun'), ('아빠', 'Noun'), ('월급', 'Noun'), ('이', 'Josa'), ('조금', 'Noun'), ('밖에', 'Josa'), ('안', 'VerbPrefix'), ('들어왔는데', 'Verb'), ('다음', 'Noun'), ('달', 'Noun'), 

In [10]:
# 클래스를 숫자로 변환
#class 0; 협박 대화
#class 1; 갈취 대화
#class 2; 직장 내 괴롭힘 대화
#class 3; 기타 괴롭힘 대화
class_label = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}

def class_to_number(conversation):
    res = []
    for element in conversation:
        res.append(class_label[element])
    return res 

In [11]:
labeled_conversation = class_to_number(train_data['class'])
print(len(labeled_conversation))

3950


In [13]:
CLASS_NUM = 5
word_counter = []
for i in range(CLASS_NUM):
    word_counter.append({})

for label, tokens in zip(labeled_conversation, tokenized_conversation):
    for token in tokens:
        # 조사와 문장부호, 길이기 1인 단어 삭제
        if token[1] == "Josa" or token[1] == "Punctuation" or len(token[0]) == 1:
            continue
        # 나머지 품사에 대해 통계
        if token[0] in word_counter[label]:
            word_counter[label][token[0]] += 1
        else:
            word_counter[label][token[0]] = 1

In [14]:
# 클래스별 많이 나오는 단어 TOP 10
class_label_inverted = {0: '협박 대화',1: '갈취 대화',2: '직장 내 괴롭힘 대화', 3: '기타 괴롭힘 대화',4: '일반 대화'}

for i in range(CLASS_NUM):
    print(f'class: {i} - {class_label_inverted[i]}')
    sorted_dic = sorted(word_counter[i], key=lambda x: word_counter[i][x], reverse=True)
    for index, data in enumerate(sorted_dic):
        print(data)
        if index > 10:
            break

class: 0 - 협박 대화
제발
지금
진짜
우리
사람
그래
무슨
그냥
죽여
죄송합니다
아니
그럼
class: 1 - 갈취 대화
진짜
그럼
아니
지금
그래
없어
내놔
그냥
제발
무슨
없어요
우리
class: 2 - 직장 내 괴롭힘 대화
죄송합니다
대리
회사
아니
지금
오늘
부장
무슨
그럼
사람
우리
그래
class: 3 - 기타 괴롭힘 대화
진짜
아니
그래
우리
그냥
무슨
지금
그렇게
아니야
사람
죄송합니다
너무
class: 4 - 일반 대화
